## Section 3: Distributed Memory Assembly of Matrix and Right-Hand Side Vector 
- using [PartitionedMatrices](https://github.com/fverdugo/PartitionedArrays.jl); need to study the example described at [example](https://www.francescverdugo.com/PartitionedArrays.jl/dev/usage/).  

<div>
<img src="./figures/dag-fem-1d.jpg" width=400 /> 
<center> Figure 1: Directed acyclic graph representation of decomposed mesh of 4 elements on the interval. A finite element discretization is assumed. </center>   
</div>

[Au = f example here.](https://www.francescverdugo.com/PartitionedArrays.jl/dev/examples/#Distributed-sparse-linear-solve)

First generate the row partition

In [1]:
# using Distributed
using PartitionedArrays
using IterativeSolvers
using LinearAlgebra

np = 3
n = 5
ranks = LinearIndices((np,))
row_partition = uniform_partition(ranks,n)

3-element Vector{PartitionedArrays.LocalIndicesWithConstantBlockSize{1}}:
 [1]
 [2, 3]
 [4, 5]

Compute the rhs vector

In [2]:
IV = map(row_partition) do row_indices
    I,V = Int[], Float64[]
    for global_row in local_to_global(row_indices)
        if global_row == 1
            v = 1.0
        elseif global_row == n
            v = -1.0
        else
            continue
        end
        push!(I,global_row)
        push!(V,v)
    end
    I,V
end
I,V = tuple_of_arrays(IV)
b = pvector!(I,V,row_partition) |> fetch

5-element PVector{Vector{Float64}} partitioned into 3 parts


Compute the system matrix

In [3]:
IJV = map(row_partition) do row_indices
    I,J,V = Int[], Int[], Float64[]
    for global_row in local_to_global(row_indices)
        if global_row in (1,n)
            push!(I,global_row)
            push!(J,global_row)
            push!(V,1.0)
        else
            push!(I,global_row)
            push!(J,global_row-1)
            push!(V,-1.0)
            push!(I,global_row)
            push!(J,global_row)
            push!(V,2.0)
            push!(I,global_row)
            push!(J,global_row+1)
            push!(V,-1.0)
        end
    end
    I,J,V
end
I,J,V = tuple_of_arrays(IJV)
col_partition = row_partition
A = psparse!(I,J,V,row_partition,col_partition) |> fetch

5×5 PSparseMatrix{SparseArrays.SparseMatrixCSC{Float64, Int64}} partitioned into 3 parts


Generate an initial guess that fulfills the boundary conditions. Solve and check the result

In [4]:
x = similar(b,axes(A,2))
x .= b
IterativeSolvers.cg!(x,A,b)
r = A*x - b
norm(r)

3.1401849173675503e-16